# Парсинг данных с сайта

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Парсинг

In [8]:
url = "https://ru.wikipedia.org/wiki/250_лучших_фильмов_по_версии_IMDb"
resp = requests.get(url)
soup = BeautifulSoup(resp.text, "html.parser")
movies_inf = soup.find_all("tr")[1:]

## Извлечение данных из таблицы

In [11]:
movies_data = []
for row in movies_inf:
    if len(cells := row.find_all("td")) < 5:
        continue
        
    url = f"https://ru.wikipedia.org/wiki/{cells[1].text.strip().replace(' ', '_')}"
    resp = requests.get(url)
    desc = (BeautifulSoup(resp.text, "html.parser").find("p").text.strip() 
            if resp.status_code == 200 else "Описание недоступно")
    
    movies_data.append({
        'Место': cells[0].text.strip(),
        'Название': cells[1].text.strip(),
        'Год': cells[2].text.strip(),
        'Режиссёр': cells[3].text.strip(),
        'Рейтинг': cells[4].text.strip(),
        'Описание': desc
    })
print("Parsing end.")

Parsing end.


## Создание датасета


In [14]:
df = pd.DataFrame(movies_data)

df.set_index('Место', inplace=True)
df.head(10)

,Название,Год,Режиссёр,Рейтинг,Описание
Место,,,,,
1,Побег из Шоушенка,1994,Фрэнк Дарабонт,"эпический фильм, историческая драма, тюремная ...",«Побе́г из Шоуше́нка» (англ. The Shawshank Red...
2,Крёстный отец,1972,Фрэнсис Форд Коппола,"эпический фильм, гангстерский фильм, фильм-тра...",Крёстный оте́ц может означать:
3,Тёмный рыцарь,2008,Кристофер Нолан,"эпический боевик, эпический фильм, супергеройс...",«Тёмный ры́царь» (англ. The Dark Knight) — аме...
4,Крёстный отец 2,1974,Фрэнсис Форд Коппола,"эпический фильм, гангстерский фильм, фильм-тра...",«Крёстный оте́ц 2» (англ. The Godfather Part I...
5,12 разгневанных мужчин,1957,Сидни Люмет,"юридическая драма, психологическая драма, дете...",«12 разгневанных мужчин» (англ. Twelve Angry M...
6,Властелин колец: Возвращение короля,2003,Питер Джексон,"приключенческая киноэпопея, эпический фильм, ф...",«Властели́н коле́ц: Возвраще́ние короля́» (анг...
7,Список Шиндлера,1993,Стивен Спилберг,"докудрама, эпический фильм, историческая киноэ...",«Спи́сок Ши́ндлера» (англ. Schindler's List) —...
8,Криминальное чтиво,1994,Квентин Тарантино,"чёрная комедия, фильм о наркопреступлении, ган...",«Кримина́льное чти́во» (англ. Pulp Fiction — «...
9,Властелин колец: Братство Кольца,2001,Питер Джексон,"приключенческая киноэпопея, эпический фильм, ф...",«Властели́н коле́ц: Бра́тство Кольца́» (англ. ...


## Вывод результатов


In [17]:
df.info(250)

<class 'pandas.core.frame.DataFrame'>
Index: 250 entries, 1 to 250
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Название  250 non-null    object
 1   Год       250 non-null    object
 2   Режиссёр  250 non-null    object
 3   Рейтинг   250 non-null    object
 4   Описание  250 non-null    object
dtypes: object(5)
memory usage: 11.7+ KB


In [21]:
df.to_csv('imdb_rating.csv')